<a href="https://colab.research.google.com/github/swaminaathakrishnan/Cool_Route_prototype/blob/master/notebooks/Cool_route_v9.1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🚀 **CoolRide V6: Cloud-Native Thermal Routing Engine**
### *Automated. Resilient. Web-Ready.*

---

### **🌍 Project Overview**
CoolRide V6 is the **Production-Ready** version of our thermal routing prototype. It creates a "Digital Twin" of Singapore's urban heat profile—modeling building shadows, tree canopies, and water cooling effects—to find the safest path for cyclists.

Unlike previous versions, V6 is **Cloud-Native**. It runs the heavy AI computation in the cloud (Colab) and automatically pushes the result to a live, serverless web dashboard that anyone can access instantly.

---

### **🔥 What's New in V6? (The "Integration Update")**

1.  **☁️ GitOps "Handshake":**
    * **Old Way:** Download KML -> Run Local Server.
    * **V6 Way:** The Python Engine automatically **commits & pushes** the route to GitHub. The Web App updates instantly.

2.  **🇸🇬 True SGT Timezone:**
    * **Problem:** Previous versions ran on UTC (Colab server time), causing "Night" shadows at noon.
    * **Fix:** V6 forces **Asia/Singapore (UTC+8)** timing. Shadows are now physically accurate for the specific ride time.

3.  **🛡️ Bulletproof Data Fetching:**
    * **Problem:** "Dirty" data in OpenStreetMap (NaN errors) would crash the water analysis.
    * **Fix:** V6 uses a **"Split-Fetch" Strategy**. It fetches Lakes, Canals, and Rivers separately. If one layer is corrupt, it is skipped, ensuring the engine *never* crashes.

4.  **🌐 Serverless Command Center:**
    * Replaces the complex local .NET setup with a lightweight **GitHub Pages Dashboard**.
    * **Live Demo Link:** [Click Here to View CoolRide Live](https://swaminaathakrishnan.github.io/Cool_Route_prototype/)

---

### **🏗️ System Architecture**

`[ 🧠 Python AI Engine ]`  --->  `[ ☁️ GitHub Repo (Storage) ]`  --->  `[ 📱 Web Dashboard (Frontend) ]`

1.  **Input:** User defines Start/End in Module 2.
2.  **Process:** Engine calculates Shadows, Trees, Water, and AI Weather trends.
3.  **Deploy:** Module 6 pushes `latest_route.kml` to the repository.
4.  **Visualize:** The Web Dashboard fetches the new file via CDN.

---

### **🏃 How to Run the Demo (Dec 30 Protocol)**

**Step 1: Authenticate**
* Ensure you have added your `GITHUB_TOKEN` to the Colab Secrets (Key icon on the left).

**Step 2: Configure Ride**
* Go to **Module 2**.
* Set your `START_POINT` and `END_POINT`.
* Set `DEPARTURE_TIME` (Default is Current Singapore Time).

**Step 3: Launch**
* Click **Runtime -> Run All**.
* Watch the logs for: `✅ CLOUD SYNC COMPLETE`.

**Step 4: Reveal**
* Open the [Live Dashboard](https://swaminaathakrishnan.github.io/Cool_Route_prototype/).
* Click **"Check for Updates"**.
* Show the Green Route to the judges.

---

### **👥 Credits**
* **Swaminaatha Krishnan:** System Architect & Cloud Integration
* **Arishya Jindal:** Algorithm Lead (Shadows & Time-Aware Routing)

In [37]:
# ==========================================
# 🧱 MODULE 1: SYSTEM INITIALIZATION
# ==========================================
# Objective: Install geospatial libraries and set up the environment.
# dependencies: OSMnx (Maps), GeoPandas (Spatial Data), Scikit-Learn (AI).

!pip install osmnx simplekml geopandas shapely networkx requests scikit-learn -q

import osmnx as ox
import networkx as nx
import simplekml
import geopandas as gpd
import pandas as pd
import requests
import numpy as np
import os
import pickle
import time
from shapely.geometry import Point, LineString, box
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from datetime import datetime, timedelta

print("✅ System Initialized. Ready for V5 Execution.")

✅ System Initialized. Ready for V5 Execution.


### ⚙️ Module 2: Configuration & Cloud Connection
Objective: Define the pilot zone and connect to the GitHub Data Lake. Logic: Instead of local files, we stream GeoJSON/CSV directly from the raw GitHub URLs. This allows the team to collaborate without sharing Drive folders.

In [38]:
# ==========================================
# ⚙️ MODULE 2: CONFIGURATION (V5.3 - AUTOMATION READY)
# ==========================================
import pytz
from datetime import datetime

print("🚴 COOLRIDE V5.3 - PRODUCTION ENGINE")
print("=" * 50)

# 1. ROUTE PARAMETERS (Bridge Variables)
# Default: Tampines Loop
START_NAME = "Tampines MRT"
START_COORDS = (1.3533, 103.9452)

END_NAME = "Tampines Eco Green"
END_COORDS = (1.3598, 103.9351)

# 2. TIME CONFIGURATION (CRITICAL FIX: UTC+8)
# We force the timezone to Asia/Singapore so shadows are accurate.
sgt_zone = pytz.timezone('Asia/Singapore')
current_time_sgt = datetime.now(sgt_zone)

# Manual Override for Demo (Optional)
# DEPARTURE_TIME = current_time_sgt.replace(hour=14, minute=0)

# Force 2:00 PM today for shadow simulation
DEPARTURE_TIME = current_time_sgt.replace(hour=14, minute=0, second=0)
# DEPARTURE_TIME = current_time_sgt

PLACE_NAME = f"{START_NAME} to {END_NAME}"
START_POINT = START_COORDS
END_POINT = END_COORDS

print(f"   📍 Route: {PLACE_NAME}")
print(f"   🕐 Departure Time (SGT): {DEPARTURE_TIME.strftime('%I:%M %p')}")
print(f"      (Sun position calculated for: {DEPARTURE_TIME})")

# 3. GITHUB DATA LAKE
GITHUB_USER = "swaminaathakrishnan"
REPO_NAME = "Cool_Route_prototype"
BASE_URL = f"https://raw.githubusercontent.com/{GITHUB_USER}/{REPO_NAME}/master/data/"

# File Links
PCN_URL = BASE_URL + "ParkConnectorLoop.geojson"
HAWKER_URL = BASE_URL + "HawkerCentresGEOJSON.geojson"
TREES_URL = BASE_URL + "trees.csv"
URA_WATER_URL = BASE_URL + "URA_Waterbody.geojson"

# AMENITIES
HAWKER_URL = BASE_URL + "hawker_centres.geojson"
SUPERMARKET_URL = BASE_URL + "supermarkets.geojson"

# 4. THERMAL WEIGHTS
WEIGHT_PCN = 0.5
WEIGHT_WATER = 0.55
WEIGHT_TREE_SHADE = 0.6
WEIGHT_BUILDING_SHADE = 0.7
WEIGHT_ULTIMATE = 0.35 # All factors combined

# 5. WATER PARAMETERS
WATER_BUFFER_DISTANCE = 100
MIN_WATER_SIZE = 50000

# 6. SAFETY OVERRIDE
NEA_HEATWAVE_ALERT = False

print("=" * 50)

🚴 COOLRIDE V5.3 - PRODUCTION ENGINE
   📍 Route: Tampines MRT to Tampines Eco Green
   🕐 Departure Time (SGT): 02:00 PM
      (Sun position calculated for: 2025-12-24 14:00:00.151450+08:00)


### Module 3A: Sun Position & Building Shadow Engine


Objective: Calculate real-time building shadows based on sun position and building heights.

**Physics:**
- Sun elevation angle determines shadow length
- Shadow direction is opposite of sun azimuth
- Taller buildings cast longer shadows (especially at low sun angles)

In [39]:

# MODULE 3A: SUN POSITION & BUILDING SHADOWS (V4)

def calculate_sun_position(latitude, longitude, timestamp):
    """
    Calculate sun elevation and azimuth for given location and time

    Returns: (elevation_degrees, azimuth_degrees)
    """
    import math
    day_of_year = timestamp.timetuple().tm_yday

    # Declination angle
    declination = 23.45 * math.sin(math.radians((360/365) * (day_of_year - 81)))

    # Hour angle
    hour = timestamp.hour + timestamp.minute / 60.0
    hour_angle = 15 * (hour - 12)

    # Sun elevation
    lat_rad = math.radians(latitude)
    dec_rad = math.radians(declination)
    ha_rad = math.radians(hour_angle)

    sin_elev = (math.sin(lat_rad) * math.sin(dec_rad) +
                math.cos(lat_rad) * math.cos(dec_rad) * math.cos(ha_rad))
    elevation = math.degrees(math.asin(max(-1, min(1, sin_elev))))

    # Sun azimuth
    cos_azim = ((math.sin(dec_rad) - math.sin(lat_rad) * sin_elev) /
                (math.cos(lat_rad) * math.cos(math.radians(elevation))))
    cos_azim = max(-1, min(1, cos_azim))
    azimuth = math.degrees(math.acos(cos_azim))

    if hour > 12:
        azimuth = 360 - azimuth

    return elevation, azimuth


def create_shadow_polygon(building_polygon, building_height, sun_elevation, sun_azimuth):
    """
    Create shadow polygon from building footprint

    Args:
        building_polygon: Shapely Polygon
        building_height: Height in meters
        sun_elevation: Sun angle above horizon (degrees)
        sun_azimuth: Sun compass direction (degrees)

    Returns:
        Shadow polygon (Shapely)
    """
    import math
    from shapely.affinity import translate

    if sun_elevation <= 0:
        return None  # Night time

    # Shadow length = height / tan(elevation)
    shadow_length = building_height / math.tan(math.radians(sun_elevation))

    # Shadow direction (opposite of sun)
    shadow_direction = (sun_azimuth + 180) % 360

    # Calculate offset in meters
    shadow_offset_y = shadow_length * math.cos(math.radians(shadow_direction))
    shadow_offset_x = shadow_length * math.sin(math.radians(shadow_direction))

    # Get building centroid
    centroid = building_polygon.centroid
    lat, lon = centroid.y, centroid.x

    # Convert meters to degrees
    deg_per_meter_lat = 1 / 111000
    deg_per_meter_lon = 1 / (111000 * math.cos(math.radians(lat)))

    offset_lat = shadow_offset_y * deg_per_meter_lat
    offset_lon = shadow_offset_x * deg_per_meter_lon

    # Create shadow by translating building polygon
    shadow = translate(building_polygon, xoff=offset_lon, yoff=offset_lat)

    # Union with building for full coverage
    full_shadow = building_polygon.union(shadow).convex_hull

    return full_shadow

print("Sun Position & Building Shadow Functions Loaded")

Sun Position & Building Shadow Functions Loaded


### Module 3B: Enhanced Spatial Graph Engine

Objective: Build road network and overlay ALL cooling features.

**Layer 1:** Road Network (OSM)  
**Layer 2:** Park Connectors (PCN)  
**Layer 3:** Tree Canopy (SGTrees)  
**Layer 4:** Building Shadows (Time-dependent)  
**Layer 5:** Water Bodies (NEW in V5! - Proximity-based cooling)

Logic: Roads receive cumulative discounts based on shade coverage from multiple sources.

In [40]:
# ==========================================
# 🗺️ MODULE 3B: ENHANCED SPATIAL GRAPH ENGINE (V7.6 - STABILITY FIX)
# ==========================================

def generate_cool_routes():
    print(f"⏳ Downloading road network for {PLACE_NAME}...")

    # 1. GET GRAPH
    try:
        G = ox.graph_from_point(START_POINT, dist=2000, network_type='bike')
        nodes = ox.graph_to_gdfs(G, edges=False)
        miny, maxy = nodes.y.min(), nodes.y.max()
        minx, maxx = nodes.x.min(), nodes.x.max()
        print(f"   📐 Zone Limits: Lat[{miny:.4f}, {maxy:.4f}], Lon[{minx:.4f}, {maxx:.4f}]")
    except Exception as e:
        print(f"   ❌ Network Error: {e}")
        return None, None, None, None, []

    # 2. LOAD PCN
    print("⏳ Overlaying Park Connectors...")
    pcn_union = None
    try:
        # Ensure data folder exists
        if not os.path.exists('data'): os.makedirs('data')

        pcn_data = gpd.read_file(PCN_URL)
        if pcn_data.crs != "EPSG:4326": pcn_data = pcn_data.to_crs("EPSG:4326")
        try: pcn_union = pcn_data.geometry.union_all()
        except: pcn_union = pcn_data.geometry.unary_union
        print(f"   ✅ PCN Loaded")
    except:
        print("   ⚠️ PCN Data missing")

    # 3. LOAD TREES
    print("⏳ Loading Tree Canopy Data...")
    trees_buffer = None
    try:
        local_file = "data/trees.csv" # Saved in data folder
        if not os.path.exists(local_file):
            lfs_url = f"https://github.com/{GITHUB_USER}/{REPO_NAME}/raw/master/data/trees.csv"
            os.system(f"wget -O {local_file} {lfs_url}")

        if os.path.exists(local_file):
            trees_df = pd.read_csv(local_file)
            trees_df.columns = [c.lower() for c in trees_df.columns]
            lat_col = 'latitude' if 'latitude' in trees_df.columns else 'lat'
            lng_col = 'longitude' if 'longitude' in trees_df.columns else 'lng'

            trees_df = trees_df[
                (trees_df[lat_col] >= miny) & (trees_df[lat_col] <= maxy) &
                (trees_df[lng_col] >= minx) & (trees_df[lng_col] <= maxx)
            ]

            if len(trees_df) > 0:
                geometry = [Point(xy) for xy in zip(trees_df[lng_col], trees_df[lat_col])]
                trees_gdf = gpd.GeoDataFrame(trees_df, geometry=geometry, crs="EPSG:4326")
                trees_buffer = trees_gdf.geometry.buffer(0.0001).unary_union
                print(f"   ✅ Tree shade layer generated ({len(trees_df)} trees)")
    except Exception as e:
        print(f"   ⚠️ Tree Data Error: {e}")

    # 4. LOAD BUILDINGS
    print("⏳ Loading Buildings...")
    building_shadows = None
    try:
        buildings_gdf = ox.features_from_point(START_POINT, tags={'building': True}, dist=2000)
        buildings_gdf = buildings_gdf[buildings_gdf.geometry.type == 'Polygon']
        buildings_gdf['estimated_height'] = 15

        sun_elev, sun_azim = calculate_sun_position(START_POINT[0], START_POINT[1], DEPARTURE_TIME)
        print(f"   ☀️ Sun (SGT): {sun_elev:.1f}° elev, {sun_azim:.1f}° azim")

        if sun_elev > 0:
            shadow_polygons = []
            for _, building in buildings_gdf.iterrows():
                shadow = create_shadow_polygon(building.geometry, building['estimated_height'], sun_elev, sun_azim)
                if shadow: shadow_polygons.append(shadow)

            from shapely.ops import unary_union
            building_shadows = unary_union(shadow_polygons)
            print(f"   ✅ Building shadow layer generated")
        else:
            print("   🌙 Night time (No shadows)")
    except Exception as e:
        print(f"   ⚠️ Building Error: {e}")

    # 5. LOAD WATER (URA) - WITH CORRUPTION FIX
    print("⏳ Loading Water Bodies...")
    water_buffer = None

    # Define Local Path & URL
    local_water = "data/URA_Waterbody.geojson"
    # Ensure we use the RAW link
    URA_WATER_URL = f"https://raw.githubusercontent.com/{GITHUB_USER}/{REPO_NAME}/master/data/URA_Waterbody.geojson"

    # 🛡️ FIX 1: Ensure directory exists
    if not os.path.exists('data'): os.makedirs('data')

    # Helper to download
    def download_ura():
        print("      📥 Downloading Official URA Data...")
        resp = requests.get(URA_WATER_URL, timeout=15)
        if resp.status_code == 200:
            with open(local_water, 'wb') as f: f.write(resp.content)
            return True
        else:
            print(f"      ⚠️ Download Failed: HTTP {resp.status_code}")
            return False

    try:
        # A. Download if missing
        if not os.path.exists(local_water):
            download_ura()

        # B. Try Loading
        try:
            water_gdf = gpd.read_file(local_water)
        except Exception as e:
            # 🛡️ FIX 2: If load fails, file is likely corrupt (HTML). Delete and retry.
            print(f"      ⚠️ File corrupted ({e}). Deleting and re-downloading...")
            if os.path.exists(local_water): os.remove(local_water)

            if download_ura():
                water_gdf = gpd.read_file(local_water) # Retry load
            else:
                water_gdf = None

        # C. Process if valid
        if water_gdf is not None:
            # Reproject if needed
            if water_gdf.crs and water_gdf.crs.to_string() != "EPSG:4326":
                water_gdf = water_gdf.to_crs("EPSG:4326")

            # Clip to Zone
            water_gdf = water_gdf.cx[minx:maxx, miny:maxy]

            if not water_gdf.empty:
                water_buffer = water_gdf.geometry.buffer(0.00075).unary_union
                print(f"   ✅ URA Water Layer Active ({len(water_gdf)} features)")
            else:
                print(f"   ℹ️ URA Data Loaded, but no water in this specific zone.")
        else:
             print("   ⚠️ Could not load URA data.")

    except Exception as e:
        print(f"   ⚠️ Water Error: {e}")

    # 6. LOAD AMENITIES
    print("⏳ Loading Amenities (Hawkers & Supermarkets)...")
    amenities = []

    def load_poi(url, type_label, name_cols):
        try:
            # Load directly from URL (GeoPandas handles this well)
            gdf = gpd.read_file(url)
            gdf = gdf.cx[minx:maxx, miny:maxy]

            count = 0
            for _, row in gdf.iterrows():
                name = "Unknown"
                for col in name_cols:
                    if col in row and pd.notna(row[col]):
                        name = row[col]
                        break
                amenities.append((name, row.geometry.y, row.geometry.x, type_label))
                count += 1
            print(f"      Found {count} {type_label}")
        except Exception as e:
            print(f"      ⚠️ Failed to load {type_label}: {e}")

    load_poi(HAWKER_URL, "Hawker", ['Name', 'NAME', 'Description'])
    load_poi(SUPERMARKET_URL, "Supermarket", ['LIC_NAME', 'STR_NAME', 'Name'])

    # 7. CALCULATE COST
    print("⏳ Calculating Costs...")
    hour = DEPARTURE_TIME.hour
    shade_multiplier = 0.6 if (hour < 10 or hour > 16) else 1.0

    for u, v, k, data in G.edges(keys=True, data=True):
        if 'geometry' in data: edge_geom = data['geometry']
        else: edge_geom = LineString([(G.nodes[u]['x'], G.nodes[u]['y']), (G.nodes[v]['x'], G.nodes[v]['y'])])

        cost = data['length']
        tags = []

        is_pcn = pcn_union and edge_geom.intersects(pcn_union)
        is_tree = trees_buffer and edge_geom.intersects(trees_buffer)
        is_shadow = building_shadows and edge_geom.intersects(building_shadows)
        is_water = water_buffer and edge_geom.intersects(water_buffer)

        if is_tree and is_shadow and is_water:
            cost *= WEIGHT_ULTIMATE
            tags.append("🌳🏢💧 Ultimate Cool")
        elif is_tree and is_shadow:
            cost *= 0.45
            tags.append("🌳🏢 Tree+Bldg")
        elif is_water:
            cost *= WEIGHT_WATER
            tags.append("💧 Water Breeze")
        elif is_tree:
            cost *= WEIGHT_TREE_SHADE
            tags.append("🌳 Tree Shade")
        elif is_shadow:
            cost *= (WEIGHT_BUILDING_SHADE * shade_multiplier)
            tags.append("🏢 Bldg Shadow")
        elif is_pcn:
            cost *= WEIGHT_PCN
            tags.append("🌿 PCN")

        data['cool_cost'] = cost
        data['tag'] = " + ".join(tags) if tags else "☀️ Exposed"

    # 8. SOLVE
    orig = ox.distance.nearest_nodes(G, START_POINT[1], START_POINT[0])
    dest = ox.distance.nearest_nodes(G, END_POINT[1], END_POINT[0])

    try:
        r_fast = nx.shortest_path(G, orig, dest, weight='length')
        r_cool = nx.shortest_path(G, orig, dest, weight='cool_cost')
        return G, r_fast, r_cool, amenities
    except Exception as e:
        print(f"   ❌ Routing failed: {e}")
        return None, None, None, None, []

In [41]:
# ==========================================
# 🧠 MODULE 4: AI PREDICTION ENGINE (V8.0 - DIURNAL CYCLES)
# ==========================================
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures

print("🧠 LOADING AI THERMAL MODEL (DIURNAL V8)...")

def predict_smart_wbgt(current_val, hour_of_day):
    """
    Predicts WBGT using a Diurnal Cycle Model (Physics-Informed AI).
    Instead of a straight line, it models the day's heat curve.
    """
    # 1. GENERATE SYNTHETIC TRAINING DATA (The "Knowledge Base")
    # We teach the AI what a "Normal Singapore Day" looks like
    hours = np.array([0, 6, 9, 12, 14, 17, 20, 23]).reshape(-1, 1)
    # Typical WBGT profile: Cool night, spike morning, peak afternoon, cool evening
    typical_profile = np.array([26.0, 25.5, 29.0, 32.5, 33.0, 31.0, 29.0, 27.0])

    # 2. FIT A POLYNOMIAL CURVE (The "Wave")
    # Degree 4 polynomial captures the double-curve of day/night
    poly = PolynomialFeatures(degree=4)
    X_poly = poly.fit_transform(hours)
    model = LinearRegression()
    model.fit(X_poly, typical_profile)

    # 3. PREDICT FOR USER'S TIME
    user_hour = np.array([[hour_of_day]])
    base_prediction = model.predict(poly.transform(user_hour))[0]

    # 4. APPLY "REALITY CORRECTION" (The "Live AI" part)
    # The model knows the 'shape' of the day, but the Sensor knows the 'actual height'.
    # If sensor says it's 35°C but model expects 33°C, we shift the whole curve up.

    # Calculate what the model *thinks* it should be right now
    # (Simplified: we assume sensor reading is 'now')
    offset = current_val - base_prediction

    # The final prediction applies this offset to the curve
    # Let's predict the heat 1 hour from now (Duration of ride)
    future_hour = np.array([[(hour_of_day + 1) % 24]])
    future_val = model.predict(poly.transform(future_hour))[0] + offset

    return future_val, offset

# Test it
# val, offset = predict_smart_wbgt(31.0, 14)
# print(f"Predicted WBGT in 1 hour: {val:.2f}°C")

🧠 LOADING AI THERMAL MODEL (DIURNAL V8)...


### 🚀 Module 5: Execution & Safe-Pace Recommendations

Objective: Synthesize map, weather, and AI data into a KML route. Upgrade:

* Govt Override: Checks NEA_HEATWAVE_ALERT.
* Safe Pacing: Suggests specific ride speeds and hydration intervals based on WBGT (ISO 7243 standards).

In [42]:
# ==========================================
# 🚀 MODULE 5: EXECUTION ENGINE (V7.5 - AMENITIES SUPPORT)
# ==========================================
import math
import os
import time
import requests
import simplekml

print("🚀 STARTING COOLRIDE ENGINE (WITH ACTIVE AI)...")

# --- HELPER: DYNAMIC SENSOR FINDER ---
def get_nearest_wbgt_station(lat, lon):
    print("⏳ Connecting to NEA Official WBGT Sensor Network...")
    url = "https://api-open.data.gov.sg/v2/real-time/api/weather"
    try:
        resp = requests.get(url, params={"api": "wbgt"}, timeout=10)
        data = resp.json()
        readings = data['data']['records'][0]['item'].get('readings', [])

        closest_station = "Unknown"
        min_dist = float('inf')
        current_val = None

        for r in readings:
            try:
                loc = {}
                s_name = "Unknown"
                if 'location' in r: loc = r['location']
                elif 'station' in r and 'location' in r['station']: loc = r['station']['location']
                if 'station' in r: s_name = r['station'].get('name', 'Unknown')
                s_lat = float(loc.get('latitude', 0))
                s_lon = float(loc.get('longitude', loc.get('longtitude', 0)))
                if s_lat == 0 or s_lon == 0: continue

                val = r.get('wbgt') or r.get('value')
                if val is None: continue
                val = float(val)
                dist = math.sqrt((lat - s_lat)**2 + (lon - s_lon)**2)

                if dist < min_dist:
                    min_dist = dist
                    closest_station = s_name
                    current_val = val
            except: continue

        if current_val is None: return 30.0, "System Fallback"
        print(f"   📍 Nearest Sensor: {closest_station} (Dist: {min_dist*111:.2f} km)")
        return current_val, closest_station
    except Exception as e:
        print(f"   ⚠️ WBGT Sensor Fail: {e}. Using Default Safety Value.")
        return 30.0, "System Fallback"

# 1. GENERATE ROUTES (Now returns 'amenities' instead of just 'shelters')
graph, r1, r2, amenities = generate_cool_routes()

if graph:
    # 2. GET WEATHER
    current_wbgt, station_name = get_nearest_wbgt_station(START_POINT[0], START_POINT[1])

    # 3. RUN AI
    pred_wbgt, trend, confidence = predict_trend(station_name, current_wbgt)
    effective_wbgt = max(current_wbgt, pred_wbgt)
    if NEA_HEATWAVE_ALERT: effective_wbgt = 35.0

    # 4. REPORT
    if effective_wbgt < 29: rec = "✅ Safe to Ride."
    elif effective_wbgt < 31: rec = "⚠️ CAUTION: Seek shade."
    else: rec = "🛑 HIGH RISK: Stop."

    print(f"\n📊 REPORT: {station_name}")
    print(f"   Current: {current_wbgt}°C | Forecast: {pred_wbgt:.1f}°C")

    # 5. EXPORT KML
    kml = simplekml.Kml()

    def add_route(route, color, name, description):
        ls = kml.newlinestring(name=name)
        coords = []
        for u, v in zip(route[:-1], route[1:]):
            d = graph.get_edge_data(u, v)[0]
            if 'geometry' in d:
                xs, ys = d['geometry'].xy
                coords.extend(list(zip(xs, ys)))
            else:
                coords.append((graph.nodes[u]['x'], graph.nodes[u]['y']))
                coords.append((graph.nodes[v]['x'], graph.nodes[v]['y']))
        ls.coords = coords
        ls.style.linestyle.color = color
        ls.style.linestyle.width = 5
        ls.description = description

    # 🧠 FUZZY LOGIC
    def check_similarity(route_a, route_b):
        set_a = set(route_a)
        set_b = set(route_b)
        intersection = len(set_a.intersection(set_b))
        union = len(set_a.union(set_b))
        return intersection / union

    sim_score = check_similarity(r1, r2)
    print(f"   🔍 Route Similarity Score: {sim_score*100:.1f}%")

    if sim_score > 0.90:
        print("   💡 Insight: Routes are effectively identical (Merged).")
        add_route(r2, simplekml.Color.green, "🌟 Recommended Route",
                  f"<b>Smart Choice</b><br>The fastest path is also the coolest.<br>Temp: {effective_wbgt:.1f}°C")
    else:
        print("   💡 Insight: A distinct cooler detour exists.")
        add_route(r1, simplekml.Color.red, "⚡ Fastest Route (Exposed)",
                  f"<b>Direct Path</b><br>Shortest time, but higher heat exposure.<br>Temp: {effective_wbgt:.1f}°C")
        add_route(r2, simplekml.Color.green, "🌿 Cool Route (Shaded)",
                  f"<b>Shaded Detour</b><br>Maximized tree canopy coverage.<br>Lower heat stress.<br>Temp: {effective_wbgt:.1f}°C")

    # --- UPDATED AMENITIES SECTION ---
    # Smart filtering: Only add points close to the Green Route (within ~300m)
    from shapely.geometry import Point, LineString

    # Create a geometric line for the cool route to check distances
    cool_route_coords = []
    for u, v in zip(r2[:-1], r2[1:]):
        d = graph.get_edge_data(u, v)[0]
        if 'geometry' in d:
            xs, ys = d['geometry'].xy
            cool_route_coords.extend(list(zip(xs, ys)))
        else:
            cool_route_coords.append((graph.nodes[u]['x'], graph.nodes[u]['y']))
            cool_route_coords.append((graph.nodes[v]['x'], graph.nodes[v]['y']))

    route_geom = LineString(cool_route_coords)

    count_amenities = 0
    if amenities:
        # Unpack 4 values now: Name, Lat, Lon, Type
        for name, lat, lon, type_label in amenities:
            poi_point = Point(lon, lat)

            # Distance check (0.003 deg is approx 300m)
            if route_geom.distance(poi_point) < 0.003:
                p = kml.newpoint(name=f"{type_label}: {name}", coords=[(lon, lat)])

                # Set Icon based on Type
                if type_label == "Hawker":
                    p.style.iconstyle.icon.href = 'http://googleusercontent.com/maps.google.com/mapfiles/kml/shapes/dining.png'
                    p.description = "<b>Hawker Centre</b><br>Cheap food & shelter."
                else:
                    p.style.iconstyle.icon.href = 'http://googleusercontent.com/maps.google.com/mapfiles/kml/shapes/grocery.png'
                    p.description = "<b>Supermarket</b><br>Water & supplies."

                count_amenities += 1

    print(f"   🍱 Added {count_amenities} amenities near the route.")

    # 6. SAVE
    if not os.path.exists('output'): os.makedirs('output')
    constant_filename = "output/latest_route.kml"
    kml.save(constant_filename)

    print(f"\n🎉 SUCCESS! Download '{constant_filename}'")
else:
    print("❌ Critical Error: Route Generation Failed.")

🚀 STARTING COOLRIDE ENGINE (WITH ACTIVE AI)...
⏳ Downloading road network for Tampines MRT to Tampines Eco Green...
   📐 Zone Limits: Lat[1.3353, 1.3713], Lon[103.9273, 103.9632]
⏳ Overlaying Park Connectors...
   ✅ PCN Loaded
⏳ Loading Tree Canopy Data...


/tmp/ipython-input-1133146671.py:57: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  trees_buffer = trees_gdf.geometry.buffer(0.0001).unary_union
/tmp/ipython-input-1133146671.py:57: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  trees_buffer = trees_gdf.geometry.buffer(0.0001).unary_union


   ✅ Tree shade layer generated (29766 trees)
⏳ Loading Buildings...
   ☀️ Sun (SGT): 51.7° elev, 227.8° azim
   ✅ Building shadow layer generated
⏳ Loading Water Bodies...


/tmp/ipython-input-1133146671.py:138: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  water_buffer = water_gdf.geometry.buffer(0.00075).unary_union
/tmp/ipython-input-1133146671.py:138: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  water_buffer = water_gdf.geometry.buffer(0.00075).unary_union


   ✅ URA Water Layer Active (44 features)
⏳ Loading Amenities (Hawkers & Supermarkets)...
      Found 3 Hawker
      Found 23 Supermarket
⏳ Calculating Costs...
⏳ Connecting to NEA Official WBGT Sensor Network...
   📍 Nearest Sensor: Bedok North Street 2 (Dist: 3.12 km)
   ⚡ Memory Hit! Loaded 128 points.

📊 REPORT: Bedok North Street 2
   Current: 25.0°C | Forecast: 25.5°C
   🔍 Route Similarity Score: 18.5%
   💡 Insight: A distinct cooler detour exists.
   🍱 Added 8 amenities near the route.

🎉 SUCCESS! Download 'output/latest_route.kml'


In [25]:
# ==========================================
# 📤 MODULE 6: CLOUD SYNC (V6.1 - SMART FIX)
# ==========================================
def push_to_github(filename):
    print("\n☁️ INITIATING CLOUD SYNC...")

    # 1. RETRIEVE TOKEN
    try:
        from google.colab import userdata
        token = userdata.get('GITHUB_TOKEN')
    except:
        token = input("   Enter GitHub PAT (Token): ")

    if not token:
        print("   ❌ Sync Failed: No Token provided.")
        return

    # 2. SETUP
    repo_url = f"https://{token}@github.com/{GITHUB_USER}/{REPO_NAME}.git"
    user_email = "coolride.bot@gmail.com"
    user_name = "CoolRide Bot"

    # 3. SMART COMMANDS (Auto-detects branch + Creates folder)
    import subprocess

    # Helper to run commands safely
    def run_git(cmd):
        try:
            # We filter out the token from errors so it doesn't leak in logs
            result = subprocess.run(cmd, shell=True, check=True, capture_output=True, text=True)
            return True, result.stdout
        except subprocess.CalledProcessError as e:
            error_msg = e.stderr.replace(token, "HIDDEN_TOKEN") # Safety mask
            print(f"   ⚠️ Git Error: {error_msg}")
            return False, error_msg

    print("   ⏳ Cloning repository...")
    run_git(f"rm -rf temp_repo") # Clean start
    success, _ = run_git(f"git clone {repo_url} temp_repo")

    if not success: return

    print("   ⏳ Processing files...")
    # Smart Move: Ensure folder exists, Copy file, Configure Git, Push to HEAD (Current Branch)
    commands = [
        f"mkdir -p temp_repo/output", # Force create folder
        f"cp {filename} temp_repo/output/latest_route.kml",
        f"cp {filename} temp_repo/output/{os.path.basename(filename)}",
        f"cd temp_repo && git config user.email '{user_email}'",
        f"cd temp_repo && git config user.name '{user_name}'",
        "cd temp_repo && git add .",
        f"cd temp_repo && git commit -m '🤖 AI Update: {time.strftime('%H:%M')}'",
        "cd temp_repo && git push origin HEAD" # Pushes to whatever branch (main/master) is active
    ]

    for cmd in commands:
        success, _ = run_git(cmd)
        if not success:
            print("   ❌ Sync Aborted due to error above.")
            return

    print("   ✅ CLOUD SYNC COMPLETE.")
    print(f"   🌐 View: https://github.com/{GITHUB_USER}/{REPO_NAME}/blob/master/output/latest_route.kml")

# EXECUTE
if 'constant_filename' in locals():
    push_to_github(constant_filename)


☁️ INITIATING CLOUD SYNC...
   ⏳ Cloning repository...
   ⏳ Processing files...
   ✅ CLOUD SYNC COMPLETE.
   🌐 View: https://github.com/swaminaathakrishnan/Cool_Route_prototype/blob/master/output/latest_route.kml


In [44]:
# ==========================================
# 🌐 MODULE 7: LIVE API SERVER (V7.9 - TIMED & SYNCED)
# ==========================================
app = Flask(__name__)
CORS(app)

@app.route('/calculate_route', methods=['POST'])
def handle_route_request():
    import time
    start_timer = time.time() # ⏱️ START CLOCK

    global START_POINT, END_POINT, DEPARTURE_TIME, PLACE_NAME

    try:
        data = request.json
        print(f"\n📨 NEW REQUEST: {data}")

        start_text = data.get('start', 'Tampines MRT')
        end_text = data.get('end', 'Tampines Eco Green')
        stop_text = data.get('stop', '')
        time_text = data.get('time', '14:00')

        # 1. Geocode
        try:
            START_POINT = ox.geocode(start_text + ", Singapore")
            END_POINT = ox.geocode(end_text + ", Singapore")
            STOP_POINT = None
            if stop_text:
                if "," in stop_text:
                    lat, lon = map(float, stop_text.split(","))
                    STOP_POINT = (lat, lon)
                else:
                    STOP_POINT = ox.geocode(stop_text + ", Singapore")
        except:
            return jsonify({"status": "error", "message": "Location not found."}), 400

        # 2. Time
        hour, minute = map(int, time_text.split(':'))
        sgt_zone = pytz.timezone('Asia/Singapore')
        DEPARTURE_TIME = datetime.now(sgt_zone).replace(hour=hour, minute=minute, second=0)

        # 3. Engine
        print("   ⚙️ Running AI Engine...")
        graph, _, _, amenities = generate_cool_routes()
        if graph is None: return jsonify({"status": "error", "message": "Graph failed."}), 500

        # Solver Helper
        def solve_path(start_node, end_node):
            try:
                r_fast = nx.shortest_path(graph, start_node, end_node, weight='length')
                r_cool = nx.shortest_path(graph, start_node, end_node, weight='cool_cost')
                return r_fast, r_cool
            except:
                return [], []

        orig_node = ox.distance.nearest_nodes(graph, START_POINT[1], START_POINT[0])
        dest_node = ox.distance.nearest_nodes(graph, END_POINT[1], END_POINT[0])

        # 4. PATH SOLVING & MERGE CHECK
        if STOP_POINT:
            print("   🛑 Multi-stop Route Detected!")
            stop_node = ox.distance.nearest_nodes(graph, STOP_POINT[1], STOP_POINT[0])
            l1_fast, l1_cool = solve_path(orig_node, stop_node)
            l2_fast, l2_cool = solve_path(stop_node, dest_node)

            if l1_fast and l2_fast:
                r1 = l1_fast[:-1] + l2_fast
                r2 = l1_cool[:-1] + l2_cool
            else:
                r1, r2 = [], []
        else:
            r1, r2 = solve_path(orig_node, dest_node)

        if not r1: return jsonify({"status": "error", "message": "No path found."}), 500

        # ⚖️ CONVERGENCE CHECK (The "Same Temp" Fix)
        # Check if routes are identical (using set logic on nodes)
        sim_score = len(set(r1).intersection(set(r2))) / len(set(r1).union(set(r2)))
        routes_converged = sim_score > 0.95

        # --- AI FORECAST ---
        current_wbgt, station_name = get_nearest_wbgt_station(START_POINT[0], START_POINT[1])
        ride_hour = DEPARTURE_TIME.hour
        forecast_wbgt, heat_offset = predict_smart_wbgt(current_wbgt, ride_hour)

        # Insight Logic
        if heat_offset > 1.0:
            insight = f"Unusually hot (+{heat_offset:.1f}°C). The AI strongly recommends the shaded route."
            status_color = "red"
        elif ride_hour >= 11 and ride_hour <= 15:
            insight = "Peak sun intensity detected. Solar radiation is maximized."
            status_color = "orange"
        else:
            insight = "Conditions are cooling down. Perfect time for a ride."
            status_color = "green"

        # 5. Generate KML
        kml = simplekml.Kml()

        def add_kml_line(route, color, name, desc):
            ls = kml.newlinestring(name=name)
            coords = []
            for u, v in zip(route[:-1], route[1:]):
                d = graph.get_edge_data(u, v)[0]
                if 'geometry' in d:
                    xs, ys = d['geometry'].xy
                    coords.extend([(float(x), float(y)) for x, y in zip(xs, ys)])
                else:
                    coords.append((float(graph.nodes[u]['x']), float(graph.nodes[u]['y'])))
                    coords.append((float(graph.nodes[v]['x']), float(graph.nodes[v]['y'])))
            ls.coords = coords
            ls.style.linestyle.color = color
            ls.style.linestyle.width = 5
            ls.description = desc
            return ls, coords

        # Dynamic Descriptions
        temp_cool = forecast_wbgt - 1.5
        if routes_converged:
            # If routes merged, show SAME temperature
            desc_fast = f"<b>Fastest Route</b><br>Temp: {forecast_wbgt:.1f}°C<br>Note: Routes Merged"
            desc_cool = f"<b>Cool Route</b><br>Temp: {forecast_wbgt:.1f}°C<br>Note: Optimized path is the same."
            temp_cool = forecast_wbgt # Reset for UI consistency
        else:
            desc_fast = f"<b>Fastest Route</b><br>Temp: {forecast_wbgt:.1f}°C<br>Exposure: High"
            desc_cool = f"<b>Cool Route</b><br>Temp: {temp_cool:.1f}°C (Est)<br>Shade: Optimized"

        add_kml_line(r1, simplekml.Color.red, "Fastest Route", desc_fast)
        _, cool_coords = add_kml_line(r2, simplekml.Color.green, "Cool Route", desc_cool)

        # 6. Amenities
        from shapely.geometry import Point, LineString
        route_geom = LineString(cool_coords)

        count = 0
        for name, lat, lon, type_label in amenities:
            lat, lon = float(lat), float(lon)
            poi_point = Point(lon, lat)

            if route_geom.distance(poi_point) < 0.003:
                emoji = "🍜" if type_label == "Hawker" else "🛒"
                p = kml.newpoint(name=f"{emoji} {name}")
                p.coords = [(lon, lat)]

                safe_name = name.replace("'", "").replace('"', "")
                p.description = f"""
                <b>{type_label}</b><br>{name}<br><br>
                <button onclick="parent.addStop('{lat},{lon}', '{safe_name}')"
                style="background:#2563eb;color:white;border:none;padding:5px 10px;cursor:pointer;border-radius:4px;">
                ➕ Add Stop Here
                </button>
                """
                count += 1

        end_timer = time.time()
        duration = end_timer - start_timer
        print(f"⏱️ Generation Time: {duration:.2f}s") # ⏱️ LOG TIME

        return jsonify({
            "status": "success",
            "kml_data": kml.kml(),
            "meta": {"duration": f"{duration:.2f}s"}, # Send time to frontend
            "ai_data": {
                "current_temp": f"{current_wbgt:.1f}",
                "forecast_temp": f"{forecast_wbgt:.1f}",
                "insight": insight,
                "color": status_color
            }
        })

    except Exception as e:
        print(f"Server Error Traceback: {e}")
        return jsonify({"status": "error", "message": str(e)}), 500

public_url = ngrok.connect(5000).public_url
print(f"🚀 SERVER ONLINE! API URL: {public_url}")
app.run(port=5000)

🚀 SERVER ONLINE! API URL: https://neurally-submucronate-sonny.ngrok-free.dev
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [23/Dec/2025 17:57:43] "OPTIONS /calculate_route HTTP/1.1" 200 -



📨 NEW REQUEST: {'start': 'Tampines MRT', 'end': 'Tampines Eco Green', 'time': '14:00', 'stop': ''}
   ⚙️ Running AI Engine...
⏳ Downloading road network for Tampines MRT to Tampines Eco Green...
   📐 Zone Limits: Lat[1.3350, 1.3710], Lon[103.9274, 103.9633]
⏳ Overlaying Park Connectors...
   ✅ PCN Loaded
⏳ Loading Tree Canopy Data...


/tmp/ipython-input-1133146671.py:57: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  trees_buffer = trees_gdf.geometry.buffer(0.0001).unary_union
/tmp/ipython-input-1133146671.py:57: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  trees_buffer = trees_gdf.geometry.buffer(0.0001).unary_union


   ✅ Tree shade layer generated (30062 trees)
⏳ Loading Buildings...
   ☀️ Sun (SGT): 51.7° elev, 227.8° azim
   ✅ Building shadow layer generated
⏳ Loading Water Bodies...
   ✅ URA Water Layer Active (44 features)
⏳ Loading Amenities (Hawkers & Supermarkets)...


...

   ✅ Tree shade layer generated (30062 trees)
⏳ Loading Buildings...
   ☀️ Sun (SGT): 51.7° elev, 227.8° azim
   ✅ Building shadow layer generated
⏳ Loading Water Bodies...


/tmp/ipython-input-1133146671.py:138: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  water_buffer = water_gdf.geometry.buffer(0.00075).unary_union
/tmp/ipython-input-1133146671.py:138: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  water_buffer = water_gdf.geometry.buffer(0.00075).unary_union


   ✅ URA Water Layer Active (44 features)
⏳ Loading Amenities (Hawkers & Supermarkets)...
      Found 3 Hawker
      Found 23 Supermarket
⏳ Calculating Costs...
   🛑 Multi-stop Route Detected!
⏳ Connecting to NEA Official WBGT Sensor Network...


INFO:werkzeug:127.0.0.1 - - [23/Dec/2025 18:02:39] "POST /calculate_route HTTP/1.1" 200 -


   📍 Nearest Sensor: Bedok North Street 2 (Dist: 3.09 km)
⏱️ Generation Time: 20.80s
